## Credits

This notebook and this code is a fork of Abid Ali Awan's tutorial on [Fine-Tuning DeepSeek R1](https://www.datacamp.com/tutorial/fine-tuning-deepseek-r1-reasoning-model). Eternally grateful for his work with DataCamp and his contributions to the community.


## Which tools & packages will we be using today?

Packages we're going to be using throughout this walkthrough will be

- `unsloth`: Efficient fine-tuning and inference for LLMs — Specifically we will be using:
    - `FastLanguageModel` module to optimize inference & fine-tuning
    - `get_peft_model` to enable LoRa (Low-Rank Adaptation) fine-tuning
- `peft`: Supports LoRA-based fine-tuning for large models.
- Different Hugging Face modules:
    - `transformers` from HuggingFace to work with our fine-tuning data and handle different model tasks
    - `trl` Transformer Reinforcement Learning from HuggingFace which allows for supervised fine-tuning of the model — we will use the `SFFTrainer` wrapper
    - `datasets` to fetch reasoning datasets from the Hugging Face Hub
- `torch`: Deep learning framework used for training
- `wandb`: Provides access to weights and biases for tracking our fine-tuning experiment

## Before we get started — how to access the Hugging Face and Weights & Biases API

### Set GPU accelerator
We are using Kaggle Notebooks because we have access to free GPUs. To enable GPU access, press on Settings > Accelerator > GPU T4 x2

### How to access the Hugging Face API

1. Register to Huggin Face if you have not already
2. Go to [Hugging Face Tokens](https://huggingface.co/settings/tokens).
3. Click **"New Token"**.
4. Select **read/write** permissions if needed.
5. Copy your **API key**.

### Weights & Biases API key**
1. Sign up at [Weights & Biases](https://wandb.ai/site).
2. Go to [W&B Settings](https://wandb.ai/settings).
3. Copy your **API key** from the "API Keys" section.

### Add the API keys to Kaggle Notebooks
1. Press on Add-ons > Secrets
2. Add the API keys under `Hugging_Face_Token` and `wnb` respectively

You can now use this code to retrieve your API keys

```py
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hugging_face_token = user_secrets.get_secret("Hugging_Face_Token")
wnb_token = user_secrets.get_secret("wnb")
```

## Install relevant packages

In [1]:
%%capture

!pip install unsloth # install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git # Also get the latest version Unsloth!

## Import all relevant packages throughout this walkthrough

In [11]:
# Modules for fine-tuning
from unsloth import FastLanguageModel
import torch # Import PyTorch
from trl import SFTTrainer # Trainer for supervised fine-tuning (SFT)
from unsloth import is_bfloat16_supported # Checks if the hardware supports bfloat16 precision
# Hugging Face modules
from huggingface_hub import login # Lets you login to API
from transformers import TrainingArguments # Defines training hyperparameters
from datasets import load_dataset # Lets you load fine-tuning datasets
# Import weights and biases
import wandb
# Import kaggle secrets
from kaggle_secrets import UserSecretsClient

## Create API keys and login to Hugging Face and Weights and Biases

In [12]:
# Initialize Hugging Face & WnB tokens
user_secrets = UserSecretsClient() # from kaggle_secrets import UserSecretsClient
hugging_face_token = user_secrets.get_secret("Hugging_Face_Token")
wnb_token = user_secrets.get_secret("wnb")

# Login to Hugging Face
login(hugging_face_token) # from huggingface_hub import login

# Login to WnB
wandb.login(key=wnb_token) # import wandb
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on Medical COT Dataset_YouTube Walkthrough', 
    job_type="training", 
    anonymous="allow"
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


## Loading DeepSeek R1 and the Tokenizer

**What are we doing in this step?**

In this step, we **load the DeepSeek R1 model and its tokenizer** using `FastLanguageModel.from_pretrained()`. We also **configure key parameters** for efficient inference and fine-tuning. We will be using a distilled 8B version of R1 for faster computation.  

**Key parameters explained**
```py
max_seq_length = 4096  # Define the maximum sequence length a model can handle (i.e., number of tokens per input)
dtype = None  # Default data type (usually auto-detected)
load_in_4bit = True  # Enables 4-bit quantization – a memory-saving optimization
```

**Intuition behind 4-bit quantization**

Imagine compressing a **high-resolution image** to a smaller size—**it takes up less space but still looks good enough**. Similarly, **4-bit quantization reduces the precision of model weights**, making the model **smaller and faster while keeping most of its accuracy**. Instead of storing precise **32-bit or 16-bit numbers**, we compress them into **4-bit values**. This allows **large language models to run efficiently on consumer GPUs** without needing massive amounts of memory.

In [13]:
# Set parameters
max_seq_length = 4096 # Define the maximum sequence length a model can handle (i.e. how many tokens can be processed at once)
dtype = None # Set to default 
load_in_4bit = True # Enables 4 bit quantization — a memory saving optimization 

# Load the DeepSeek R1 model and tokenizer using unsloth — imported using: from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",  # Load the pre-trained DeepSeek R1 model (8B parameter version)
    max_seq_length=max_seq_length, # Ensure the model can process up to 2048 tokens at once
    dtype=dtype, # Use the default data type (e.g., FP16 or BF16 depending on hardware support)
    load_in_4bit=load_in_4bit, # Load the model in 4-bit quantization to save memory
    token=hugging_face_token, # Use hugging face token
)

==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

## Testing DeepSeek R1 on a medical use-case before fine-tuning


### Defining a system prompt 
To create a prompt style for the model, we will define a system prompt and include placeholders for the question and response generation. The prompt will guide the model to think step-by-step and provide a logical, accurate response.

In [16]:
# Define a system prompt under prompt_style 
prompt_style = """以下是一个描述任务的指令，同时提供了相关背景信息。
请根据问题生成恰当的回答。在回答前，请逐步思考问题，确保逻辑严谨且答案准确。

### 指令:
您是一位精通临床推理、诊断和治疗方案的医学专家。
请回答以下医学问题。

### 问题:
{}

### 回答:
<think>{}"""

### Running inference on the model

In this step, we **test the DeepSeek R1 model** by providing a **medical question** and generating a response.  
The process involves the following steps:

1. **Define a test question** related to a medical case.
2. **Format the question using the structured prompt (`prompt_style`)** to ensure the model follows a logical reasoning process.
3. **Tokenize the input and move it to the GPU (`cuda`)** for faster inference.
4. **Generate a response using the model**, specifying key parameters like `max_new_tokens=1200` (limits response length).
5. **Decode the output tokens back into text** to obtain the final readable answer.

In [17]:
# Creating a test medical question for inference
question = """一位61岁的女性，长期在咳嗽或打喷嚏等活动中不自主排尿，但夜间无漏尿，现接受妇科检查和棉签测试。
根据这些发现，膀胱测压最有可能揭示她的残余尿量和逼尿肌收缩情况吗？"""

# Enable optimized inference mode for Unsloth models (improves speed and efficiency)
FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!

# Format the question using the structured prompt (`prompt_style`) and tokenize it
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")  # Convert input to PyTorch tensor & move to GPU

# Generate a response using the model
outputs = model.generate(
    input_ids=inputs.input_ids, # Tokenized input question
    attention_mask=inputs.attention_mask, # Attention mask to handle padding
    max_new_tokens=1200, # Limit response length to 1200 tokens (to prevent excessive output)
    use_cache=True, # Enable caching for faster inference
)

# Decode the generated output tokens into human-readable text
response = tokenizer.batch_decode(outputs)

# Extract and print only the relevant response part (after "### Response:")
print(response[0].split("### 回答:")[1])


<think>
嗯，我现在需要解决一个关于膀胱测压的问题。让我先仔细阅读题目，理解背景和要求。

题目说，一位61岁的女性，长期在咳嗽或打喷嚏时不自主排尿，但夜间没有漏尿。现在她正在接受妇科检查和棉签测试。问题问的是，根据这些发现，膀胱测压最有可能揭示她的残余尿量和逼尿肌收缩情况吗？

首先，我要分析她的症状。她长期在咳嗽或打喷嚏时不自主排尿，这可能意味着她的膀胱在这些活动中受到了刺激，导致不自主排尿。夜间没有漏尿，说明她夜间的排尿是自主的，没有尿失禁的情况。这可能表明她白天的不自主排尿可能是由于膀胱的某种反射活动，比如膀胱的反射性收缩，或者是膀胱的不完全空缩。

接下来，考虑膀胱测压的作用。膀胱测压通常用于评估膀胱的压力和容积，常用于诊断尿潴留、膀胱功能障碍等。测压时，医生会通过直肠镜或导管将压力传感器插入膀胱，测量膀胱的最大容积和压力。此外，测压还可以评估膀胱的舒张性和收缩性。

残余尿量是指膀胱在排尿后仍然保留的尿液量，通常由超声或其他影像学检查确定。但是题目提到的是膀胱测压是否能揭示残余尿量。实际上，膀胱测压主要测量膀胱的容积和压力，并不是直接测量残余尿量，残余尿量更多依赖于排尿前后的尿液量测定或超声检查。

至于逼尿肌收缩情况，膀胱测压可以间接反映膀胱的状态。膀胱的舒张和收缩与脊髓中的低级中枢有关，尤其是S2-S3神经节。膀胱测压可以帮助评估膀胱的舒张性和收缩性，如果膀胱过度收缩，可能导致反射性排尿困难，从而引发不自主排尿的情况。因此，膀胱测压可以提供关于膀胱功能的信息，帮助诊断是否存在膀胱肌肉的异常收缩。

结合题目中的患者，她长期在咳嗽或打喷嚏时不自主排尿，可能提示膀胱的反射性活动异常，而夜间无漏尿表明夜间排尿是正常的。这可能与膀胱的反射性收缩有关，膀胱测压可以帮助评估膀胱的收缩情况，进而判断是否存在膀胱肌肉的异常。因此，膀胱测压最有可能揭示她的残余尿量和逼尿肌收缩情况。

总结一下，膀胱测压主要用于评估膀胱的容积和压力，而不是直接测量残余尿量。但是，它可以间接反映膀胱的收缩情况，帮助诊断是否存在膀胱肌肉的异常收缩，进而解释患者的不自主排尿现象。因此，膀胱测压在这方面是有帮助的。
</think>

根据患者的症状和膀胱测压的作用，以下是逐步的解释和答案：

1. **患者症状分析**：
   - 患者在咳嗽或打喷嚏时不自主排尿，但夜间无漏尿。这表明

>**Before starting fine-tuning — why are we fine-tuning in the first place?**
>
> Even without fine-tuning, our model successfully generated a chain of thought and provided reasoning before delivering the final answer. The reasoning process is encapsulated within the `<think>` `</think>` tags. So, why do we still need fine-tuning? The reasoning process, while detailed, was long-winded and not concise. Additionally, we want the final answer to be consistent in a certain style.

## Fine-tuning step by step

## Step 1 — Update the system prompt 
We will slightly change the prompt style for processing the dataset by adding the third placeholder for the complex chain of thought column. `</think>`

In [18]:
# Updated training prompt style to add </think> tag 
train_prompt_style = """以下是一个描述任务的指令，同时提供了相关背景信息。
请根据问题生成恰当的回答。在回答前，请逐步思考问题，确保逻辑严谨且答案准确。

### 指令：
您是一位精通临床推理、诊断和治疗方案的医学专家。
请回答以下医学问题。

### 问题：
{}

### 回答：
<think>
{}
</think>
{}"""

## Step 2 — Download the fine-tuning dataset and format it for fine-tuning

We will use the Medical O1 Reasoninng SFT found here on [Hugging Face](https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT). From the authors: This dataset is used to fine-tune HuatuoGPT-o1, a medical LLM designed for advanced medical reasoning. This dataset is constructed using GPT-4o, which searches for solutions to verifiable medical problems and validates them through a medical verifier.

In [19]:
# Download the dataset using Hugging Face — function imported using from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","zh", split = "train[0:500]",trust_remote_code=True) # Keep only first 500 rows
dataset

README.md: 0.00B [00:00, ?B/s]

medical_o1_sft_Chinese.json:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20171 [00:00<?, ? examples/s]

Dataset({
    features: ['Question', 'Complex_CoT', 'Response'],
    num_rows: 500
})

In [20]:
# Show an entry from the dataset
dataset[1]

{'Question': '对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？',
 'Complex_CoT': "嗯，有一个60岁的男性患者，出现了右侧胸疼，而且X光显示右侧肋膈角消失，这看起来很像是胸腔积液的问题。医生诊断是肺结核伴右侧胸腔积液。那我们就得想想，什么样的实验室检查能帮助我们更好地理解这个胸水的性质呢？\n\n首先，我们得从基础知识说起。胸腔积液就是说胸腔里有了多余的液体。这液体的来源可能是感染、恶性肿瘤或者其他因素。病因有些复杂，所以了解胸水性质很关键。\n\n嗯，实验室检查一般是用来分析胸水的，看看它到底是从哪儿来的。常见的检测项包括蛋白质、乳酸脱氢酶（LDH）、葡萄糖，以及更具体的病原检测，比如结核菌。\n\n患者是肺结核，那我们就要考虑到这可能是结核性胸腔积液。先要判断胸水的类型：是漏出液还是渗出液呢？为了确定这个，我们通常会用Light's标准，这个标准可以通过胸水的蛋白浓度和LDH水平区分。\n\n但是吧，光知道是渗出液可能还不足以确认是不是结核引起的。这时，结核相关的精确检测就显得格外重要，比如说结核菌PCR或者ADA（腺苷脱氨酶）活性。\n\n对了，ADA这个指标很有意思，特别是在结核性胸腔积液中。当ADA活性高的时候，通常会提示我们大概率是结核病。因此，虽然蛋白质和LDH很重要，但要确认结核，ADA或许更直接些。\n\n也就是说，虽然最开始我们要确认胸水是渗出液，但结合患者肺结核的情况，ADA检测会给我们更好的线索。\n\n所以，经过这么一番思考，我觉得对于已经被诊断为肺结核的患者来说，检测腺苷脱氨酶（ADA）更有指引性。这能帮忙进一步确认胸水与结核感染的关联。\n\n嗯，综上所述，ADA作为分析结核性胸腔积液的指标真的非常有效，最终还是选择它来作进一步检查。",
 'Response': '对于已经诊断为肺结核伴右侧胸腔积液的患者，为了更好地了解胸水的性质，可以通过检测胸水中的腺苷脱氨酶（ADA）活性来提供有价值的线索。ADA活性高通常提示结核性胸腔积液的可能性更大，因此在这种情况下，ADA检测相较于其他指标如蛋白质和乳酸脱氢酶（LDH）而言，更具诊断意义。'}

>**Next step is to structure the fine-tuning dataset according to train prompt style—why?**
>
> - Each question is paired with chain-of-thought reasoning and the final response.
> - Ensures every training example follows a consistent pattern.
> - Prevents the model from continuing beyond the expected response lengt by adding the EOS token.

In [21]:
# We need to format the dataset to fit our prompt training style 
EOS_TOKEN = tokenizer.eos_token  # Define EOS_TOKEN which the model when to stop generating text during training
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [22]:
# Define formatting prompt function
def formatting_prompts_func(examples):  # Takes a batch of dataset examples as input
    inputs = examples["Question"]       # Extracts the medical question from the dataset
    cots = examples["Complex_CoT"]      # Extracts the chain-of-thought reasoning (logical step-by-step explanation)
    outputs = examples["Response"]      # Extracts the final model-generated response (answer)
    
    texts = []  # Initializes an empty list to store the formatted prompts
    
    # Iterate over the dataset, formatting each question, reasoning step, and response
    for input, cot, output in zip(inputs, cots, outputs):  
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN  # Insert values into prompt template & append EOS token
        texts.append(text)  # Add the formatted text to the list

    return {
        "text": texts,  # Return the newly formatted dataset with a "text" column containing structured prompts
    }

In [23]:
# Update dataset formatting
dataset_finetune = dataset.map(formatting_prompts_func, batched = True)
dataset_finetune["text"][0]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

'以下是一个描述任务的指令，同时提供了相关背景信息。\n请根据问题生成恰当的回答。在回答前，请逐步思考问题，确保逻辑严谨且答案准确。\n\n### 指令：\n您是一位精通临床推理、诊断和治疗方案的医学专家。\n请回答以下医学问题。\n\n### 问题：\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### 回答：\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。\n

In [24]:
# Apply LoRA (Low-Rank Adaptation) fine-tuning to the model 
model_lora = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank: Determines the size of the trainable adapters (higher = more parameters, lower = more efficiency)
    target_modules=[  # List of transformer layers where LoRA adapters will be applied
        "q_proj",   # Query projection in the self-attention mechanism
        "k_proj",   # Key projection in the self-attention mechanism
        "v_proj",   # Value projection in the self-attention mechanism
        "o_proj",   # Output projection from the attention layer
        "gate_proj",  # Used in feed-forward layers (MLP)
        "up_proj",    # Part of the transformer’s feed-forward network (FFN)
        "down_proj",  # Another part of the transformer’s FFN
    ],
    lora_alpha=16,  # Scaling factor for LoRA updates (higher values allow more influence from LoRA layers)
    lora_dropout=0,  # Dropout rate for LoRA layers (0 means no dropout, full retention of information)
    bias="none",  # Specifies whether LoRA layers should learn bias terms (setting to "none" saves memory)
    use_gradient_checkpointing="unsloth",  # Saves memory by recomputing activations instead of storing them (recommended for long-context fine-tuning)
    random_state=3407,  # Sets a seed for reproducibility, ensuring the same fine-tuning behavior across runs
    use_rslora=False,  # Whether to use Rank-Stabilized LoRA (disabled here, meaning fixed-rank LoRA is used)
    loftq_config=None,  # Low-bit Fine-Tuning Quantization (LoFTQ) is disabled in this configuration
)

Unsloth 2025.8.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Step 3 — Setting up the model using LoRA

**An intuitive explanation of LoRA** 

Large language models (LLMs) have **millions or even billions of weights** that determine how they process and generate text. When fine-tuning a model, we usually update all these weights, which **requires massive computational resources and memory**.

LoRA (**Low-Rank Adaptation**) allows to fine-tune efficiently by:

- Instead of modifying all weights, **LoRA adds small, trainable adapters** to specific layers.  
- These adapters **capture task-specific knowledge** while leaving the original model unchanged.  
- This reduces the number of trainable parameters **by more than 90%**, making fine-tuning **faster and more memory-efficient**.  

Think of an LLM as a **complex factory**. Instead of rebuilding the entire factory to produce a new product, LoRA **adds small, specialized tools** to existing machines. This allows the factory to adapt quickly **without disrupting its core structure**.

For a more technical explanation, check out this tutorial by [Sebastian Raschka](https://www.youtube.com/watch?v=rgmJep4Sb4&t).

Below, we will use the `get_peft_model()` function which stands for Parameter-Efficient Fine-Tuning — this function wraps the base model (`model`) with LoRA modifications, ensuring that only specific parameters are trained.

Now, we initialize `SFTTrainer`, a supervised fine-tuning trainer from `trl` (Transformer Reinforcement Learning), to fine-tune our model efficiently on a dataset.

In [25]:
# Initialize the fine-tuning trainer — Imported using from trl import SFTTrainer
trainer = SFTTrainer(
    model=model_lora,  # The model to be fine-tuned
    tokenizer=tokenizer,  # Tokenizer to process text inputs
    train_dataset=dataset_finetune,  # Dataset used for training
    dataset_text_field="text",  # Specifies which field in the dataset contains training text
    max_seq_length=max_seq_length,  # Defines the maximum sequence length for inputs
    dataset_num_proc=2,  # Uses 2 CPU threads to speed up data preprocessing

    # Define training arguments
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Number of examples processed per device (GPU) at a time
        gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps before updating weights
        num_train_epochs=1, # Full fine-tuning run
        warmup_steps=5,  # Gradually increases learning rate for the first 5 steps
        max_steps=60,  # Limits training to 60 steps (useful for debugging; increase for full fine-tuning)
        learning_rate=2e-4,  # Learning rate for weight updates (tuned for LoRA fine-tuning)
        fp16=not is_bfloat16_supported(),  # Use FP16 (if BF16 is not supported) to speed up training
        bf16=is_bfloat16_supported(),  # Use BF16 if supported (better numerical stability on newer GPUs)
        logging_steps=10,  # Logs training progress every 10 steps
        optim="adamw_8bit",  # Uses memory-efficient AdamW optimizer in 8-bit mode
        weight_decay=0.01,  # Regularization to prevent overfitting
        lr_scheduler_type="linear",  # Uses a linear learning rate schedule
        seed=3407,  # Sets a fixed seed for reproducibility
        output_dir="outputs",  # Directory where fine-tuned model checkpoints will be saved
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/500 [00:00<?, ? examples/s]

## Step 4 — Model training! 

This should take around 30 to 40 minutes — we can then check out our training results on Weights and Biases

In [26]:
# Start the fine-tuning process
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.079000
20,1.665600
30,1.620600
40,1.550700
50,1.527500
60,1.529500


In [27]:
# Save the fine-tuned model
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▃▁▁▁▂
train/learning_rate,█▇▅▄▂▁
train/loss,█▃▂▁▁▁
total_flos,3.95576617992192e+16
train/epoch,1.896
train/global_step,60
train/grad_norm,0.22107
train/learning_rate,0.0
train/loss,1.5295


## Step 5 — Run model inference after fine-tuning

In [28]:
question = """一位61岁的女性，长期在咳嗽或打喷嚏等活动中不自主排尿，但夜间无漏尿，
现接受妇科检查和棉签测试。根据这些发现，膀胱测压最有可能揭示她的残余尿量和逼尿肌收缩情况吗？"""

# Load the inference model using FastLanguageModel (Unsloth optimizes for speed)
FastLanguageModel.for_inference(model_lora)  # Unsloth has 2x faster inference!

# Tokenize the input question with a specific prompt format and move it to the GPU
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response using LoRA fine-tuned model with specific parameters
outputs = model_lora.generate(
    input_ids=inputs.input_ids,          # Tokenized input IDs
    attention_mask=inputs.attention_mask, # Attention mask for padding handling
    max_new_tokens=1200,                  # Maximum length for generated response
    use_cache=True,                        # Enable cache for efficient generation
)

# Decode the generated response from tokenized format to readable text
response = tokenizer.batch_decode(outputs)

# Extract and print only the model's response part after "### Response:"
print(response[0].split("### 回答:")[1])


<think>
嗯，咳嗽或者打喷嚏的时候不自主排尿，这个情况听起来有点奇怪。通常来说，咳嗽的时候会有排尿的冲动，特别是当膀胱里有尿液的时候。可是她却在这些活动中不自主排尿，这让我有点困惑。

另外，夜间没有漏尿，这个信息也挺有意思的。一般来说，如果膀胱里有很多尿液，夜间漏尿的几率比较大。她的情况不一样，说明她的膀胱里可能没有那么多尿液。

再看看她接受了妇科检查和棉签测试。这些检查应该能帮助我们了解她的膀胱状态。妇科检查可能会告诉我们膀胱的大小和形状，而棉签测试则能帮助我们了解膀胱的尿量和膀胱收缩情况。

结合这些信息，膀胱测压应该能帮助我们了解膀胱里的残余尿量和膀胱的收缩情况。测压可以告诉我们膀胱里有多少尿液，以及膀胱肌肉的状态。这样一来，我们就能更准确地了解她为什么在咳嗽或者打喷嚏的时候不自主排尿。

嗯，通过这些信息，我们可以得出膀胱测压确实是个不错的选择来了解她的残余尿量和膀胱收缩情况。看来这个思路是对的。
</think>
根据提供的信息，膀胱测压确实是一个有帮助的方法来了解您的患者的残余尿量和膀胱收缩情况。通过测量膀胱内的压力，我们可以估算膀胱的残余尿量以及膀胱肌肉的收缩状态。膀胱测压通常用于评估膀胱的容量和功能状态，这对于解释为什么在咳嗽或打喷嚏时不自主排尿以及夜间没有漏尿的现象非常有帮助。<｜end▁of▁sentence｜>


In [29]:
question = """一位59岁男性出现发热、寒战、盗汗和全身疲劳，超声发现主动脉瓣上有12mm赘生物，
血培养显示革兰阳性、触酶阴性、γ溶血性链球菌，在6.5% NaCl培养基中不生长。此患者病情最可能的诱发因素是什么？"""

# Tokenize the input question with a specific prompt format and move it to the GPU
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response using LoRA fine-tuned model with specific parameters
outputs = model_lora.generate(
    input_ids=inputs.input_ids,          # Tokenized input IDs
    attention_mask=inputs.attention_mask, # Attention mask for padding handling
    max_new_tokens=1200,                  # Maximum length for generated response
    use_cache=True,                        # Enable cache for efficient generation
)

# Decode the generated response from tokenized format to readable text
response = tokenizer.batch_decode(outputs)

# Extract and print only the model's response part after "### Response:"
print(response[0].split("### 回答:")[1])


<think>
患者是59岁男性，发热、寒战、盗汗、全身疲劳，这些症状让我想到感染性疾病，尤其是细菌感染。看到他有12mm的赘生物在主动脉瓣上，听起来像是动脉瓣炎。然后，血培养显示的是革兰阳性、触酶阴性、γ溶血性链球菌，这些特征让我联想到流行性甲型链球菌病。可是，他在6.5%的NaCl培养基中不生长，这让我有点困惑。

我想，6.5%的NaCl培养基通常是用来培养大肠杆菌的，因为它们对高浓度的盐有耐受性。既然他在这种培养基中无法生长，那么这个结果可能表明他并没有大肠杆菌感染。也就是说，虽然血培养结果看起来像是大肠杆菌，但实际上他没有感染这种细菌。

再想想，他的症状和动脉瓣赘生物，结合这些信息，可能是因为某种其他类型的细菌感染。比如，流行性甲型链球菌病虽然在传统的培养基中生长，但在高盐培养基中却无法生长。这可能意味着他的感染并非由大肠杆菌引起，而是由流行性甲型链球菌病。

所以，综合来看，他的症状、赘生物以及培养结果都指向流行性甲型链球菌病的可能性，尤其是考虑到他在6.5% NaCl培养基中无法生长，这进一步排除了大肠杆菌的可能性。
</think>
根据患者的症状和病理检查结果，患者的症状包括发热、寒战、盗汗和全身疲劳，这些都是典型的细菌感染表现。特别是他在主动脉瓣上有12mm的赘生物，这提示可能存在动脉瓣炎。血培养显示的是革兰阳性、触酶阴性、γ溶血性链球菌，这些特征通常与流行性甲型链球菌病有关。

然而，他在6.5% NaCl培养基中无法生长，这与大肠杆菌的特点不符，因为大肠杆菌通常在高盐环境中能生长。因此，患者的感染可能不是由大肠杆菌引起的，而是由流行性甲型链球菌病引起的。这种细菌在传统的培养基中能生长，而在高盐培养基中则无法生长。因此，结合这些信息，患者的感染最可能是流行性甲型链球菌病。<｜end▁of▁sentence｜>


## Saving the model locally

In [30]:
new_model_local = "DeepSeek-R1-Medical-COT-zh"
model_lora.save_pretrained(new_model_local) 
tokenizer.save_pretrained(new_model_local)

model_lora.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Unsloth: Merging weights into 16bit:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  25%|██▌       | 1/4 [00:43<02:10, 43.40s/it]

...

Unsloth: Merging weights into 16bit:  50%|█████     | 2/4 [01:24<01:24, 42.16s/it]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  75%|███████▌  | 3/4 [02:08<00:42, 42.77s/it]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [02:24<00:00, 36.07s/it]


## Pushing the model to Hugging Face Hub

new_model_online = "yangxiaomin/DeepSeek-R1-Medical-COT-zh"
model_lora.push_to_hub(new_model_online)
tokenizer.push_to_hub(new_model_online)

model_lora.push_to_hub_merged(new_model_online, tokenizer, save_method = "merged_16bit")